In [ ]:
import numpy as np
from numba import njit
import matplotlib.pyplot as plt

# ─────────────────────────────────────
# 1. Global constants & Pauli operators
# ─────────────────────────────────────
PI        = np.pi
B0     = 1.0
omega0 = 1.0*100
h_FD      = 1.0e-4
total_T   = 1.2
N_slices  = 1000
tau       = total_T / N_slices
theta_rot = PI / 3
n_epochs  = 3000
J         = 0.1

# Pauli matrices
σx = np.array([[0,1],[1,0]],complex)
σy = np.array([[0,-1j],[1j,0]],complex)
σz = np.array([[1,0],[0,-1]],complex)
I2 = np.eye(2, dtype=complex)

σ1x,σ1y,σ1z = np.kron(σx,I2),np.kron(σy,I2),np.kron(σz,I2)
ZZ          = np.kron(σz,σz)

# Bell-state projectors
bell = np.array([[1,0,0,-1],
                 [0,1,1,0],
                 [0,1,-1,0],
                 [1,0,0,1]],complex)/np.sqrt(2)
projs = np.array([np.outer(v,v.conj()) for v in bell])

ψ0  = np.array([1,0,0,1],complex)/np.sqrt(2)
ρ0  = np.outer(ψ0,ψ0.conj())

# ─────────────────────────────────────
# 2. Unitary block
# ─────────────────────────────────────
@njit(inline='always')
def U4_full(vx, vy, vz, Jcouple, dt):
    hz = np.array([vz+Jcouple, vz-Jcouple])
    U  = np.zeros((4,4),dtype=np.complex128)
    for blk in range(2):
        mag = np.sqrt(vx*vx + vy*vy + hz[blk]*hz[blk])
        #if mag < 1e-14:
        #    cs=1.0; ss=0.0
        #else:
        #    cs = np.cos(mag*dt)
        #    ss = np.sin(mag*dt)/mag

        cs = np.cos(mag*dt)
        ss = np.sin(mag*dt)/mag

        a = cs-1j*ss*hz[blk]; d = cs+1j*ss*hz[blk]
        b = -1j*ss*(vx-1j*vy); c = -1j*ss*(vx+1j*vy)
        if blk==0:
            U[0,0]=a; U[0,2]=b; U[2,0]=c; U[2,2]=d
        else:
            U[1,1]=a; U[1,3]=b; U[3,1]=c; U[3,3]=d
    return U

@njit
def probabilities(theta_rot, tau, n_slices,
                  B, ω,
                  Vx, Vy, Vz, Jcouple,
                  projs):
    ρ = ρ0.copy()
    for t in range(n_slices):
        t_now = t*tau
        # --- target fields ---
        vx_t = B * np.cos(ω*t_now)
        vy_t = 0.0
        vz_t = 0.0
        # --- control fields ---
        vx_t += Vx[t]; vy_t += Vy[t]; vz_t += Vz[t]
        U  = U4_full(vx_t, vy_t, vz_t, Jcouple, tau)
        ρ  = U @ ρ @ U.conj().T

    # measurement-axis rotation
    c = 1/np.sqrt(3)
    Urot = U4_full(c,c,c,0.0,theta_rot)
    ρ = Urot @ ρ @ Urot.conj().T

    p = np.empty(4)
    for k in range(4):
        p[k] = np.real(np.trace(ρ @ projs[k]))
    return p

@njit
def f0_and_trace(Vx,Vy,Vz,
                 theta_rot,tau,n_slice,
                 B0,ω0,h,Jcouple,projs):
    # base + ±h on B and ω
    P0   = probabilities(theta_rot,tau,n_slice,B0,   ω0,   Vx,Vy,Vz,Jcouple,projs)
    PBp  = probabilities(theta_rot,tau,n_slice,B0+h, ω0,   Vx,Vy,Vz,Jcouple,projs)
    PBm  = probabilities(theta_rot,tau,n_slice,B0-h, ω0,   Vx,Vy,Vz,Jcouple,projs)
    Pwp  = probabilities(theta_rot,tau,n_slice,B0,   ω0+h,Vx,Vy,Vz,Jcouple,projs)
    Pwm  = probabilities(theta_rot,tau,n_slice,B0,   ω0-h,Vx,Vy,Vz,Jcouple,projs)

    grad = np.empty((2,4))
    grad[0]=(PBp-PBm)/(2*h)
    grad[1]=(Pwp-Pwm)/(2*h)

    F=np.zeros((2,2))
    for i in range(2):
        for j in range(2):
            acc=0.0
            for k in range(4):
                acc += grad[i,k]*grad[j,k]/max(P0[k],1e-12)
            F[i,j]=acc
    Cov = np.linalg.inv(F)
    tr_inv = Cov[0,0]+Cov[1,1]
    f0 = 1.0/(1/F[0,0]+1/F[1,1])
    return f0,tr_inv

# ─────────────────────────────────────
# 3. Initial control: Optimal control for H0 without the Ising term
# ─────────────────────────────────────
B_ctrl   = B0
ω_ctrl   = omega0

V1x=np.zeros(N_slices); V1y=np.zeros(N_slices); V1z=np.zeros(N_slices)
for t in range(N_slices):
    t_now=t*tau
    V1x[t] = -B_ctrl*np.cos(ω_ctrl*t_now)
    V1y[t] = ω_ctrl/2
    V1z[t] = 0.0

# ─────────────────────────────────────
# 3. Initial control: Control obtained by GRAPE in the former case
# ─────────────────────────────────────
# V1x = [-1.0002423167, -0.9929081987, -0.9714045014, -0.9359968398, -0.8870749440, -0.8254416715, -0.7518690105, -0.6673946207, -0.5735917637, -0.4714487476, -0.3624261364, -0.2482357121, -0.1304332184, -0.0109161716, 0.1088765701, 0.2271477312, 0.3420223910, 0.4520549982, 0.5557834929, 0.6512082196, 0.7373032918, 0.8129143506, 0.8766376801, 0.9280329290, 0.9660443071, 0.9900804465, 0.9995805432, 0.9949325992, 0.9760999510, 0.9433140819, 0.8967404729, 0.8372651716, 0.7658174611, 0.6833196393, 0.5909768824, 0.4902373704, 0.3824362352, 0.2689548560, 0.1517113066, 0.0322231967, -0.0876798934, -0.2062588892, -0.3219748497, -0.4329255015, -0.5377204904, -0.6348807232, -0.7225802373, -0.8003084617, -0.8662480881, -0.9198589748, -0.9601756909, -0.9867256662, -0.9991088358, -0.9971212933, -0.9807892240, -0.9502443106, -0.9061856989, -0.8490274432, -0.7797075026, -0.6991600261, -0.6083563702, -0.5091219700, -0.4023976962, -0.2898755946, -0.1731954161, -0.0540444233, 0.0657884188, 0.1850357584, 0.3012490511, 0.4132102916, 0.5190640535, 0.6177004000, 0.7075404929, 0.7870430684, 0.8552981609, 0.9110967945, 0.9538268192, 0.9829705122, 0.9978813257, 0.9984823797, 0.9846699376, 0.9566420323, 0.9150057441, 0.8602101657, 0.7928116837, 0.7141254669, 0.6252188520, 0.5274679867, 0.4218492474, 0.3103763301, 0.1942141266, 0.0754746285, -0.0444675858, -0.1638076992, -0.2807065166, -0.3936196725, -0.5007515161, -0.6007191495, -0.6920826174, -0.7735732208, -0.8439921823, -0.9021091995, -0.9473013873, -0.9788022582, -0.9963439261, -0.9995798809, -0.9882723351, -0.9628513335, -0.9235683683, -0.8710143512, -0.8059363144, -0.7293413684, -0.6420196953, -0.5456786470, -0.4415830341, -0.3308113619, -0.2154746914, -0.0971759941, 0.0228769929, 0.1422017213, 0.2598142498, 0.3733459723, 0.4819384095, 0.5833029067, 0.6763200669, 0.7597900634, 0.8319528372, 0.8924218552, 0.9400448230, 0.9741117390, 0.9939996417, 0.9998787723, 0.9913180148, 0.9684075657, 0.9314016204, 0.8812783776, 0.8183652900, 0.7437182350, 0.6583540931, 0.5636029492, 0.4607165804, 0.3509617201, 0.2364531381, 0.1184034960, -0.0014376430, -0.1209456455, -0.2390070076, -0.3534309145, -0.4629606718, -0.5658000703, -0.6603222735, -0.7455869223, -0.8200212988, -0.8826143420, -0.9325595436, -0.9690484578, -0.9916153918, -1.0000703830, -0.9940989706, -0.9736159370, -0.9393258973, -0.8914594574, -0.8305865440, -0.7581736448, -0.6745530845, -0.5813157364, -0.4796578290, -0.3712757403, -0.2574499356, -0.1398413886, -0.0204190257, 0.0994154023, 0.2178052014, 0.3329723610, 0.4435131402, 0.5477242506, 0.6437994339, 0.7309604638, 0.8072802831, 0.8721999827, 0.9244002455, 0.9634778299, 0.9885529135, 0.9994860499, 0.9959080678, 0.9783322639, 0.9462584582, 0.9007326599, 0.8424563524, 0.7718368690, 0.6901977769, 0.5986639963, 0.4985100705, 0.3911389148, 0.2781474689, 0.1610737618, 0.0419482527, -0.0780358503, -0.1968986065, -0.3127404447, -0.4242480246, -0.5296508793, -0.6273776623, -0.7161063793, -0.7944763153, -0.8614406633, -0.9160777203, -0.9575392728, -0.9851419325, -0.9986366776, -0.9978610013, -0.9827310673, -0.9531420897, -0.9100702456, -0.8540477470, -0.7855433666, -0.7059366813, -0.6160542975, -0.5173243850, -0.4110897327, -0.2989144909, -0.1824368966, -0.0635772156, 0.0563614351, 0.1754926484, 0.2919329220, 0.4044344849, 0.5111692904, 0.6103858256, 0.7007111309, 0.7810035639, 0.8501588177, 0.9071750524, 0.9509411960, 0.9812063724, 0.9972081187, 0.9988593839, 0.9863090835, 0.9594823767, 0.9187486991, 0.8649153316, 0.7986866011, 0.7208227695, 0.6326458222, 0.5355603331, 0.4305552140, 0.3193464239, 0.2036083471, 0.0849735933, -0.0348900617, -0.1542621993, -0.2714326868, -0.3848755627, -0.4924902107, -0.5932037049, -0.6852017248, -0.7675196300, -0.8388375683, -0.8979108716, -0.9442336474, -0.9767169424, -0.9955198831, -0.9998167285, -0.9897271683, -0.9654104810, -0.9271996634, -0.8756493052, -0.8115245997, -0.7358697230, -0.6494349917, -0.5535256518, -0.4500550179, -0.3399450710, -0.2247600403, -0.1065275517, 0.0131634689, 0.1328922483, 0.2505706224, 0.3645586215, 0.4734651737, 0.5754735125, 0.6691847444, 0.7533583777, 0.8267221288, 0.8880275422, 0.9367414515, 0.9717410534, 0.9930895828, 0.9999623654, 0.9925601321, 0.9706894226, 0.9350108505, 0.8858078786, 0.8238201390, 0.7500539382, 0.6655432318, 0.5713275371, 0.4690218848, 0.3599529132, 0.2457280532, 0.1279954056, 0.0082100966, -0.1114361075, -0.2296215620, -0.3444690162, -0.4543314390, -0.5579283506, -0.6530044394, -0.7391408716, -0.8144394753, -0.8781825923, -0.9290730100, -0.9666476796, -0.9904482118, -0.9998262519, -0.9950603425, -0.9757375865, -0.9424598685, -0.8956654259, -0.8361919979, -0.7643771609, -0.6815375852, -0.5890325840, -0.4881605051, -0.3801021240, -0.2666684528, -0.1494197152, -0.0299069084, 0.0898418651, 0.2084998488, 0.3240706432, 0.4349628843, 0.5396791811, 0.6365339879, 0.7243757331, 0.8015043253, 0.8673270061, 0.9206354295, 0.9608508756, 0.9870311259, 0.9990568590, 0.9968420800, 0.9801585827, 0.9493829643, 0.9049885056, 0.8475240004, 0.7780068258, 0.6972085785, 0.6063385946, 0.5067796079, 0.3998990620, 0.2874331710, 0.1706526553, 0.0514010224, -0.0684530680, -0.1876234797, -0.3036302553, -0.4155515211, -0.5213501861, -0.6199809783, -0.7093352573, -0.7886045250, -0.8566308635, -0.9121571067, -0.9547442371, -0.9834794395, -0.9982741036, -0.9983378129, -0.9843441302, -0.9559495762, -0.9141161741, -0.8589437696, -0.7913970201, -0.7126427322, -0.6235160743, -0.5254640924, -0.4197096395, -0.3081372511, -0.1919130480, -0.0730176275, 0.0468125668, 0.1660805003, 0.2828516113, 0.3958332238, 0.5027357780, 0.6025267750, 0.6939175687, 0.7752792043, 0.8453315422, 0.9029766272, 0.9479178924, 0.9792604078, 0.9964725213, 0.9993793977, 0.9878187106, 0.9620744959, 0.9224289796, 0.8696803656, 0.8044253786, 0.7274205109, 0.6400868763, 0.5435586332, 0.4391661358, 0.3284565660, 0.2130040539, 0.0944001169, -0.0253937393, -0.1447944117, -0.2624010796, -0.3758616698, -0.4840924981, -0.5853096068, -0.6783360776, -0.7612692863, -0.8334209636, -0.8936153507, -0.9409586093, -0.9746973205, -0.9944661523, -0.9998648101, -0.9910237339, -0.9678361081, -0.9308785768, -0.8802233809, -0.8171037562, -0.7422891128, -0.6566160462, -0.5616475061, -0.4585419942, -0.3487980514, -0.2340786812, -0.1160111750, 0.0036817782, 0.1233437952, 0.2413026316, 0.3556823881, 0.4650337331, 0.5674759223, 0.6620890166, 0.7471223152, 0.8212563727, 0.8836413537, 0.9333899576, 0.9695389310, 0.9919884182, 0.9998269810, 0.9936576779, 0.9728251181, 0.9382717891, 0.8903128659, 0.8292851842, 0.7564084469, 0.6727459619, 0.5792421479, 0.4776284817, 0.3689096124, 0.2548991095, 0.1373013863, 0.0177588350, -0.1020041861, -0.2202634524, -0.3355615398, -0.4457805832, -0.5498824394, -0.6458310469, -0.7326760760, -0.8089823229, -0.8734336406, -0.9254327555, -0.9642392354, -0.9891211363, -0.9997101077, -0.9958863534, -0.9777549396, -0.9455996612, -0.8998894400, -0.8412034064, -0.7704416702, -0.6885817516, -0.5968798836, -0.4965656728, -0.3890856629, -0.2758447199, -0.1588358670, -0.0395257191, 0.0803914313, 0.1991298903, 0.3151271559, 0.4263032410, 0.5315993112, 0.6293448274, 0.7175662919, 0.7959520970, 0.8626684378, 0.9169580024, 0.9581175048, 0.9854491821, 0.9986541677, 0.9976417900, 0.9820566146, 0.9523328989, 0.9089210155, 0.8526631570, 0.7839085249, 0.7039255645, 0.6139923309, 0.5149605146, 0.4086997163, 0.2965248465, 0.1800267724, 0.0610449734, -0.0590539017, -0.1779961872, -0.2945931978, -0.4068257391, -0.5133019512, -0.6123812298, -0.7025473464, -0.7826408963, -0.8516509067, -0.9081723853, -0.9517644249, -0.9817207318, -0.9975904882, -0.9988405602, -0.9860148678, -0.9588021043, -0.9179919281, -0.8638538643, -0.7972770289, -0.7192577654, -0.6308321476, -0.5333825580, -0.4284187917, -0.3171922913, -0.2014130093, -0.0826026118, 0.0372488653, 0.1565850286, 0.2737493822, 0.3868759194, 0.4945285180, 0.5950090944, 0.6869368208, 0.7690405212, 0.8400158664, 0.8987544536, 0.9449192564, 0.9772020652, 0.9955993876, 0.9994184813, 0.9891820536, 0.9645082461, 0.9261503087, 0.8743801780, 0.8100402951, 0.7339285429, 0.6473337303, 0.5514236146, 0.4475806606, 0.3374841140, 0.2222997931, 0.1039912303, -0.0157698212, -0.1354846580, -0.2531171143, -0.3670590683, -0.4756919681, -0.5776086340, -0.6712987476, -0.7550991480, -0.8281752920, -0.8893153605, -0.9376963502, -0.9726653678, -0.9934916501, -0.9999923824, -0.9922315512, -0.9702367368, -0.9341241177, -0.8848088077, -0.8226176031, -0.7485464618, -0.6638199587, -0.5694702451, -0.4671626141, -0.3577212757, -0.2435116338, -0.1255869346, -0.0058814913, 0.1138188511, 0.2319980857, 0.3467199798, 0.4565452979, 0.5598181081, 0.6550105174, 0.7406237663, 0.8158042988, 0.8790220814, 0.9298328227, 0.9672960908, 0.9906501863, 0.9998700087, 0.9946143354, 0.9750634986, 0.9415383731, 0.8945776046, 0.8344350807, 0.7626284288, 0.6797262762, 0.5871339289, 0.4859222785, 0.3777458035, 0.2641349416, 0.1467751089, 0.0273677107, -0.0925252565, -0.2110988250, -0.3265031299, -0.4374655483, -0.5418450587, -0.6385810344, -0.7261864997, -0.8031668023, -0.8688706659, -0.9218557184, -0.9617243470, -0.9875511204, -0.9993206988, -0.9967631523, -0.9797028860, -0.9488316198, -0.9041316559, -0.8464096525, -0.7765783332, -0.6954411377, -0.6044204335, -0.5047773159, -0.3977259916, -0.2851165522, -0.1682858380, -0.0490892356, 0.0708250655, 0.1899004419, 0.3057010752, 0.4176310308, 0.5233831822, 0.6216298985, 0.7109100555, 0.7900323094, 0.8577056023, 0.9129328968, 0.9553774384, 0.9837509581, 0.9981403023, 0.9980300984, 0.9836771672, 0.9553279947, 0.9130802775, 0.8576207062, 0.7898659975, 0.7106494669, 0.6213474875, 0.5230342045, 0.4173210792, 0.3054961552, 0.1894523497, 0.0705425234, -0.0494473286, -0.1686770282, -0.2853680149, -0.3983077440, -0.5051235379, -0.6049362456, -0.6957187483, -0.7767216996, -0.8465528998, -0.9042030961, -0.9488191001, -0.9799252244, -0.9967931578, -0.9993627839, -0.9876538306, -0.9615073559, -0.9216856637, -0.8686593719, -0.8029711222, -0.7257621226, -0.6383040247, -0.5415288963, -0.4370226568, -0.3261619935, -0.2106406860, -0.0920873609, 0.0277288408, 0.1471308088, 0.2644980245, 0.3780699823, 0.4862065259, 0.5873203808, 0.6800424601, 0.7629623123, 0.8347310336, 0.8945649032, 0.9416077639, 0.9751066075, 0.9944899519, 0.9997316173, 0.9905755403, 0.9670058724, 0.9296962524, 0.8790156813, 0.8155925030, 0.7404619239, 0.6545755070, 0.5595170352, 0.4562801381, 0.3465213989, 0.2316604616, 0.1135113468, -0.0063739059, -0.1258785958, -0.2438368746, -0.3580923460, -0.4672660637, -0.5698305140, -0.6639282956, -0.7487949967, -0.8226937268, -0.8850129951, -0.9343531385, -0.9704173772, -0.9924057988, -0.9999569839, -0.9933544832, -0.9724342541, -0.9376941179, -0.8891201988, -0.8279947877, -0.7548024680, -0.6709362555, -0.5773424440, -0.4754897564, -0.3667000643, -0.2527082483, -0.1350600125, -0.0155034476, 0.1044424104, 0.2225413677, 0.3377623761, 0.4479679024, 0.5517248524, 0.6477621079, 0.7342462819, 0.8101626400, 0.8744626010, 0.9260894257, 0.9647674263, 0.9892637431, 0.9995154313, 0.9954611900, 0.9771818275, 0.9447346121, 0.8986619818, 0.8397452497, 0.7687947921, 0.6867063176, 0.5945796161, 0.4941844906, 0.3865656951, 0.2733701386, 0.1562442587, 0.0368450635, -0.0829682893, -0.2016905577, -0.3174625964, -0.4285586643, -0.5338240416, -0.6311289969, -0.7195548845, -0.7975132240, -0.8639565376, -0.9180866401, -0.9590276458, -0.9859617786, -0.9988857651, -0.9974240922, -0.9815977362, -0.9517991959, -0.9081258061, -0.8514497569, -0.7824677683, -0.7022502044, -0.6120281899, -0.5129798398, -0.4065619209, -0.2943108060, -0.1777211042, -0.0586900787, 0.0612930956, 0.1803760164, 0.2967445375, 0.4090784814, 0.5152945723, 0.6142191072, 0.7042223279, 0.7841978186, 0.8527371235, 0.9090567057, 0.9525324794, 0.9820612489, 0.9975318243, 0.9987637132, 0.9852836509, 0.9579468887, 0.9168101553, 0.8623677473, 0.7957332139, 0.7174522808, 0.6290332631, 0.5311997936, 0.4260419219, 0.3146671696, 0.1986206669, 0.0800536963, -0.0399386548, -0.1592644472, -0.2762288179, -0.3894010276, -0.4968656314, -0.5972809420, -0.6888687291, -0.7707697776, -0.8413782939, -0.9000377452, -0.9456773993, -0.9778868615, -0.9960291445, -0.9996771973, -0.9890529200, -0.9640648265, -0.9253939889, -0.8733330645, -0.8085782838, -0.7324800315, -0.6456529771, -0.5496349239, -0.4456009645, -0.3350666746, -0.2200705739, -0.1015917866, 0.0182701376, 0.1377679597, 0.2553376762, 0.3691174811, 0.4779067468, 0.5796941865, 0.6730031786, 0.7565703164, 0.8295244479, 0.8903491205, 0.9384811031, 0.9728934646, 0.9935704918, 0.9998677009, 0.9918518853, 0.9694392080, 0.9331903539, 0.8835054062, 0.8210649214, 0.7466989734, 0.6618032415, 0.5673504872, 0.4646940346, 0.3553728786, 0.2409340420, 0.1230085265, 0.0033200005, -0.1164226988, -0.2344161253, -0.3491809951, -0.4589185246, -0.5619535894, -0.6569587924, -0.7424971213, -0.8173126761, -0.8805004683, -0.9308901043, -0.9679308105, -0.9911416312, -0.9999921218, -0.9944188924, -0.9747997709, -0.9408591761, -0.8935026551, -0.8332748583, -0.7610840568, -0.6780187695, -0.5850715844, -0.4837619048, -0.3753823013, -0.2619358022, -0.1445604725, -0.0250007213, 0.0948618977, 0.2132761598, 0.3287548455, 0.4394365575, 0.5437999264, 0.6403532125, 0.7276672826, 0.8045322861, 0.8698573144, 0.9225748022, 0.9621142181, 0.9878342922, 0.9991732835, 0.9964525379, 0.9792391854, 0.9477942763, 0.9028479724, 0.8448878586, 0.7749494087, 0.6935555995, 0.6022360993, 0.5023431859, 0.3954140881, 0.2825823841, 0.1657791392, 0.0464489901, -0.0733831740, -0.1924419529, -0.3084585639, -0.4200273391, -0.5256233136, -0.6236653424, -0.7127112495, -0.7917019575, -0.8592054890, -0.9143009581, -0.9561391099, -0.9843839153, -0.9984043765, -0.9983234611, -0.9833700834, -0.9546753089, -0.9120079357, -0.8564186553, -0.7885175262, -0.7091335735, -0.6195206309, -0.5211619385, -0.4151978606, -0.3033388139, -0.1873202093, -0.0682217604, 0.0518212814, 0.1709979753, 0.2876779715, 0.4003635033, 0.5070919603, 0.6065309682, 0.6974463783, 0.7780626397, 0.8478394372, 0.9052248459, 0.9495307559, 0.9802620087, 0.9967700569, 0.9989581503, 0.9870385328, 0.9607956010, 0.9206023102, 0.8672800801, 0.8013620525, 0.7238859951, 0.6362822428, 0.5394172389, 0.4346652521, 0.3236788750, 0.2081923337, 0.0894943085, -0.0302692143, -0.1497926164, -0.2670834892, -0.3803618257, -0.4884349692, -0.5893522770, -0.6818569877, -0.7645891846, -0.8363231341, -0.8959938518, -0.9426994139, -0.9758761946, -0.9949095574, -0.9998113510, -0.9903661639, -0.9665931351, -0.9288262850, -0.8778907763]
# V1y = [49.9980128662, 50.0007390019, 50.0133382128, 50.0349239224, 50.0638275194, 50.0985448717, 50.1366082272, 50.1761936991, 50.2146423930, 50.2496887891, 50.2795997817, 50.3024633347, 50.3168134938, 50.3222430295, 50.3183098590, 50.3051317579, 50.2837933263, 50.2554227019, 50.2216431697, 50.1845155309, 50.1459604959, 50.1083424972, 50.0736535238, 50.0435718595, 50.0200910846, 50.0040729722, 49.9964652564, 49.9975229209, 50.0067165948, 50.0238209760, 50.0472651213, 50.0757758886, 50.1077998557, 50.1408186850, 50.1734765506, 50.2036000715, 50.2293817850, 50.2493791629, 50.2626822732, 50.2682326164, 50.2659324658, 50.2563061538, 50.2392545248, 50.2162652971, 50.1886234363, 50.1580566275, 50.1259430063, 50.0944088824, 50.0650319529, 50.0394019461, 50.0190941726, 50.0045959003, 49.9971638193, 49.9966805109, 50.0032578196, 50.0161575484, 50.0344147774, 50.0572317960, 50.0827839997, 50.1097486099, 50.1364513888, 50.1610255607, 50.1827756652, 50.1995622737, 50.2111687947, 50.2165437898, 50.2157674925, 50.2085591655, 50.1959384852, 50.1780797329, 50.1562395474, 50.1320837867, 50.1065140165, 50.0811510559, 50.0572485670, 50.0363953012, 50.0194236960, 50.0072361461, 50.0003111425, 49.9988516855, 50.0031283059, 50.0124071667, 50.0260640834, 50.0431843597, 50.0626362484, 50.0833297376, 50.1038880509, 50.1231442426, 50.1401333918, 50.1535343502, 50.1630117935, 50.1677799075, 50.1677726549, 50.1629450752, 50.1538779243, 50.1409398745, 50.1250561968, 50.1069957117, 50.0880599954, 50.0688172844, 50.0509238038, 50.0348722981, 50.0217796600, 50.0121402438, 50.0062903827, 50.0045706142, 50.0068496642, 50.0127775965, 50.0220819457, 50.0337517367, 50.0473968947, 50.0618038987, 50.0764143959, 50.0899535757, 50.1019779269, 50.1116133180, 50.1185225873, 50.1221829021, 50.1225883423, 50.1196858573, 50.1138360092, 50.1053424611, 50.0947966150, 50.0828548866, 50.0701855013, 50.0574279879, 50.0454942003, 50.0348576137, 50.0258346713, 50.0191618729, 50.0150404694, 50.0132371336, 50.0143338116, 50.0174252737, 50.0226865707, 50.0293265504, 50.0372829290, 50.0454550361, 50.0539088141, 50.0614984991, 50.0685370199, 50.0738803638, 50.0778228920, 50.0798391106, 50.0802206901, 50.0787650531, 50.0756141796, 50.0712693560, 50.0657020915, 50.0596500208, 50.0533139147, 50.0470371489, 50.0412157192, 50.0358321439, 50.0316784940, 50.0283254595, 50.0262124422, 50.0252189752, 50.0251123883, 50.0260002686, 50.0276973028, 50.0295729577, 50.0318624910, 50.0342132653, 50.0362960639, 50.0381024826, 50.0397013591, 50.0405028713, 50.0408846695, 50.0408613696, 50.0405919917, 50.0398523167, 50.0391005442, 50.0383613577, 50.0377490916, 50.0373470055, 50.0368880221, 50.0372142112, 50.0373355299, 50.0379610174, 50.0386211427, 50.0392330132, 50.0396758246, 50.0398437230, 50.0393438840, 50.0383069512, 50.0367201292, 50.0343762982, 50.0312384460, 50.0276211748, 50.0236592396, 50.0194305222, 50.0151451196, 50.0112061302, 50.0079644796, 50.0051553548, 50.0036326672, 50.0032778197, 50.0042723686, 50.0067598599, 50.0104519873, 50.0154130461, 50.0211945902, 50.0276103604, 50.0342073198, 50.0408491246, 50.0467417430, 50.0515920462, 50.0549556375, 50.0565161565, 50.0563032640, 50.0539796130, 50.0494268570, 50.0430527058, 50.0349992069, 50.0257428670, 50.0155849281, 50.0050547333, 49.9949085475, 49.9857955568, 49.9781699278, 49.9724591354, 49.9691249336, 49.9687220728, 49.9710959447, 49.9763347092, 49.9838797740, 49.9937742121, 50.0055616575, 50.0182289525, 50.0311155615, 50.0438245296, 50.0550857945, 50.0647508769, 50.0715102560, 50.0754345061, 50.0755848534, 50.0724397192, 50.0656274493, 50.0556567759, 50.0430051247, 50.0279766502, 50.0117405807, 49.9951257859, 49.9788315919, 49.9641914505, 49.9519372812, 49.9428601322, 49.9373717144, 49.9361298927, 49.9391095110, 49.9462529257, 49.9578034213, 49.9722705235, 49.9894565980, 50.0082381285, 50.0278126674, 50.0466439021, 50.0638763878, 50.0781317871, 50.0890946111, 50.0954168541, 50.0969498234, 50.0933458880, 50.0847931622, 50.0715380444, 50.0544612705, 50.0342115736, 50.0119577195, 49.9889923089, 49.9667704423, 49.9462619196, 49.9287875499, 49.9158211619, 49.9074685134, 49.9049824539, 49.9082628996, 49.9172250620, 49.9315942331, 49.9504556986, 49.9730072428, 49.9978779269, 50.0236684594, 50.0488189483, 50.0719855062, 50.0918346033, 50.1068262415, 50.1165168487, 50.1195863378, 50.1161183996, 50.1062247262, 50.0903374570, 50.0691673224, 50.0439191236, 50.0157435243, 49.9865649347, 49.9579155680, 49.9314665683, 49.9087276439, 49.8912241125, 49.8797406026, 49.8754411526, 49.8783148836, 49.8885833497, 49.9053806088, 49.9283046567, 49.9558250396, 49.9865196745, 50.0184844628, 50.0501983430, 50.0796378061, 50.1049749854, 50.1247011913, 50.1378554471, 50.1433010685, 50.1405234218, 50.1296929528, 50.1115926901, 50.0868175781, 50.0566450031, 50.0229718750, 49.9876516424, 49.9526140812, 49.9197741590, 49.8915111357, 49.8689450723, 49.8540328108, 49.8472187063, 49.8491169703, 49.8600473936, 49.8790345040, 49.9052722027, 49.9374797932, 49.9738796216, 50.0121139437, 50.0503072375, 50.0860393151, 50.1173208930, 50.1423155868, 50.1594259597, 50.1675219918, 50.1658202925, 50.1549089379, 50.1349198994, 50.1069285852, 50.0724356128, 50.0331846567, 49.9916943996, 49.9501810466, 49.9111269378, 49.8768227746, 49.8491082184, 49.8299033546, 49.8199538526, 49.8205977994, 49.8315108271, 49.8522369077, 49.8815930002, 49.9181601115, 49.9598139952, 50.0043156406, 50.0488533058, 50.0910572961, 50.1286576742, 50.1590772980, 50.1805853145, 50.1919473966, 50.1923040815, 50.1815192840, 50.1601544211, 50.1292469193, 50.0905889214, 50.0461898217, 49.9987431296, 49.9509289092, 49.9053388405, 49.8646711594, 49.8314618471, 49.8074073024, 49.7942173999, 49.7927777037, 49.8032567364, 49.8249706948, 49.8570377108, 49.8974137821, 49.9443412796, 49.9945395046, 50.0456699437, 50.0945868747, 50.1383659989, 50.1747216344, 50.2011296046, 50.2161697343, 50.2189842868, 50.2089284493, 50.1868702548, 50.1535870306, 50.1111517159, 50.0619254956, 50.0086609819, 49.9543044366, 49.9019496069, 49.8549657482, 49.8156610423, 49.7867794382, 49.7695493026, 49.7654536801, 49.7748635841, 49.7971411939, 49.8312613769, 49.8754814576, 49.9268805159, 49.9829466998, 50.0405531775, 50.0961490093, 50.1466041763, 50.1891085952, 50.2208029690, 50.2401022273, 50.2455465928, 50.2368529692, 50.2145834199, 50.1794860094, 50.1336540042, 50.0797872562, 50.0209294365, 49.9601252543, 49.9013756999, 49.8475530152, 49.8021248310, 49.7673962321, 49.7460101156, 49.7388092632, 49.7465811313, 49.7689861503, 49.8045459335, 49.8518213371, 49.9079443114, 49.9693677330, 50.0333502773, 50.0956854748, 50.1528902116, 50.2016612868, 50.2393648887, 50.2631116452, 50.2721139358, 50.2652637401, 50.2430789969, 50.2066959981, 50.1582407308, 50.0998578301, 50.0356433472, 49.9687047589, 49.9031459386, 49.8425034267, 49.7903932306, 49.7500141802, 49.7237056712, 49.7128796755, 49.7184938827, 49.7403366819, 49.7769965007, 49.8268725590, 49.8867787412, 49.9538471662, 50.0239392334, 50.0930332661, 50.1571602265, 50.2128131713, 50.2563443052, 50.2855800574, 50.2982051953, 50.2939169635, 50.2725635437, 50.2353046871, 50.1842175285, 50.1222029353, 50.0526019595, 49.9795835146, 49.9071707174, 49.8395269543, 49.7807909205, 49.7341610680, 49.7025260706, 49.6875285108, 49.6904303337, 49.7111148559, 49.7485349504, 49.8003959445, 49.8642324675, 49.9361908920, 50.0123699775, 50.0880828488, 50.1591643828, 50.2217182421, 50.2717516820, 50.3064535822, 50.3236281695, 50.3222355696, 50.3024682971, 50.2649027295, 50.2117642098, 50.1462614771, 50.0718551818, 49.9930738817, 49.9137993459, 49.8391119146, 49.7733025545, 49.7202090726, 49.6824675172, 49.6630615243, 49.6626972712, 49.6816969960, 49.7191802023, 49.7727947623, 49.8399246826, 49.9164700590, 49.9984202911, 50.0808142170, 50.1592678109, 50.2287693887, 50.2854934716, 50.3262464099, 50.3483618533, 50.3504336832, 50.3325677317, 50.2952728734, 50.2406953448, 50.1720869811, 50.0929858107, 50.0083906279, 49.9228002623, 49.8409685725, 49.7680094688, 49.7078424285, 49.6641336913, 49.6395949101, 49.6353978218, 49.6522059564, 49.6890004764, 49.7440698879, 49.8136529439, 49.8948429536, 49.9822942820, 50.0712646320, 50.1566319388, 50.2335351132, 50.2974363175, 50.3444078991, 50.3719514305, 50.3782033051, 50.3626619563, 50.3263065795, 50.2707463265, 50.1995681517, 50.1164309199, 50.0263701513, 49.9340074122, 49.8452269848, 49.7646774000, 49.6975883337, 49.6472517322, 49.6171221023, 49.6085587481, 49.6225613924, 49.6583661804, 49.7139398159, 49.7862223755, 49.8712114416, 49.9639186657, 50.0594198235, 50.1518280884, 50.2362303630, 50.3074251093, 50.3611515140, 50.3945515378, 50.4054316211, 50.3930409420, 50.3578825891, 50.3020775605, 50.2286677619, 50.1416498210, 50.0463079092, 49.9475425802, 49.8517193070, 49.7638973887, 49.6893133608, 49.6320907372, 49.5957745241, 49.5824656121, 49.5931624827, 49.6273579028, 49.6829032783, 49.7570349702, 49.8457138500, 49.9434881043, 50.0451659457, 50.1447855269, 50.2365788321, 50.3152361480, 50.3762463163, 50.4159604020, 50.4320276775, 50.4231388895, 50.3900757667, 50.3342005037, 50.2590954706, 50.1689232768, 50.0684545318, 49.9637606696, 49.8608783466, 49.7652657988, 49.6831016830, 49.6186695086, 49.5758118077, 49.5573338271, 49.5640642633, 49.5959776072, 49.6509154013, 49.7267380728, 49.8182752714, 49.9208837848, 50.0286889679, 50.1352407492, 50.2346739836, 50.3212556898, 50.3895413822, 50.4360678663, 50.4574943874, 50.4529657752, 50.4223094266, 50.3672638571, 50.2912288596, 50.1977011280, 50.0927837335, 49.9820864488, 49.8720602240, 49.7692493028, 49.6791232534, 49.6071093544, 49.5571824826, 49.5328056668, 49.5352820248, 49.5642734485, 49.6185798046, 49.6950216022, 49.7894033729, 49.8964428732, 50.0098387470, 50.1233799640, 50.2304538993, 50.3247592989, 50.4010126387, 50.4545174662, 50.4823197808, 50.4824109362, 50.4549011878, 50.4010871485, 50.3242033630, 50.2284275408, 50.1192429316, 50.0029806058, 49.8862209372, 49.7754974629, 49.6775059286, 49.5975751277, 49.5406432596, 49.5099483285, 49.5070983741, 49.5328683869, 49.5855940074, 49.6623051601, 49.7589003883, 49.8699932688, 49.9888139926, 50.1091312288, 50.2238695310, 50.3263347750, 50.4104047599, 50.4714886037, 50.5057690675, 50.5111896137, 50.4873866119, 50.4355564415, 50.3585803951, 50.2607676997, 50.1478932407, 50.0260131547, 49.9024541916, 49.7843701166, 49.6781231595, 49.5903952130, 49.5255684532, 49.4880824961, 49.4800555869, 49.5015168983, 49.5520769394, 49.6287455042, 49.7269490740, 49.8414301905, 49.9656126004, 50.0925362745, 50.2149042985, 50.3253368142, 50.4178164731, 50.4867177653, 50.5280401172, 50.5393109445, 50.5197659653, 50.4702900877, 50.3936959464, 50.2944654823, 50.1781002951, 50.0514025142, 49.9214498403, 49.7954796022, 49.6812374678, 49.5851679180, 49.5123929072, 49.4675700735, 49.4534938416, 49.4706645451, 49.5184633705, 49.5941787898, 49.6936100791, 49.8111828911, 49.9404779677, 50.0737622149, 50.2032844559, 50.3220043548, 50.4228401157, 50.5000269765, 50.5487822272, 50.5665190120, 50.5516506932, 50.5051457166, 50.4298243220, 50.3297046344, 50.2102971555, 50.0787974866, 49.9426699623, 49.8091863140, 49.6868043115, 49.5819939330, 49.5010495935, 49.4486300947, 49.4278491011, 49.4397784639, 49.4843932202, 49.5586217122, 49.6588416437, 49.7790507128, 49.9126290318, 50.0519688750, 50.1890635240, 50.3160607542, 50.4254640301, 50.5109748569, 50.5677312595, 50.5921749913, 50.5828284380, 50.5401260519, 50.4663092329, 50.3655922065, 50.2438869281, 50.1079310946, 49.9655286423, 49.8251515396, 49.6943514849, 49.5808935671, 49.4913529267, 49.4309421467, 49.4029273942, 49.4093730238, 49.4497321479, 49.5221910568, 49.6222006382, 49.7445661179, 49.8823616265, 50.0274434245, 50.1715502726, 50.3067504589, 50.4250243693, 50.5193036814, 50.5841988104, 50.6161430213, 50.6126671923, 50.5741145689, 50.5026004368, 50.4020711031, 50.2783460723, 50.1385488155, 49.9904432815, 49.8426183373, 49.7035948773, 49.5814868320, 49.4828237715, 49.4138547393, 49.3784518793, 49.3783731989, 49.4144063464, 49.4839985115, 49.5834790339, 49.7073854244, 49.8486220471, 49.9991900831, 50.1505283934, 50.2936995269, 50.4206724094, 50.5240986248, 50.5977208356, 50.6371805985, 50.6402261876, 50.6064583816, 50.5377277351, 50.4379428153, 50.3127910225, 50.1694129420, 50.0157686642, 49.8609654256, 49.7138989472, 49.5826651000, 49.4748365714, 49.3969715950, 49.3532918705, 49.3464950032, 49.3771130227, 49.4434484257, 49.5416564635, 49.6666364988, 49.8108282184, 49.9663336578, 50.1242882400, 50.2756232267, 50.4112752239, 50.5238624012, 50.6062457755, 50.6541382558, 50.6640898209, 50.6357712198, 50.5706116732, 50.4719938579, 50.3459363940, 50.1994248858, 50.0407286954, 49.8787889680, 49.7235144925, 49.5832097918, 49.4660373201, 49.3786854108, 49.3264148973, 49.3121533939, 49.3366074835, 49.3988938146, 49.4951424115, 49.6201085897, 49.7669996032, 49.9271341175, 50.0914813150, 50.2503630525, 50.3949952362, 50.5167275163, 50.6082889298, 50.6645941636, 50.6821389659, 50.6596438734, 50.5985558449, 50.5019615137, 50.3753822195, 50.2260331974, 50.0625596558, 49.8941463966, 49.7306149516, 49.5807633915, 49.4540550741, 49.3568804588, 49.2952277733, 49.2726183967, 49.2904627008, 49.3476360452, 49.4412020901, 49.5658032365, 49.7141511363, 49.8783221550, 50.0484457563, 50.2148573558, 50.3679537885, 50.4990933367, 50.6000371666, 50.6653360304, 50.6907926139, 50.6748930267, 50.6181895534, 50.5240021808, 50.3976153695, 50.2459545836, 50.0779362852, 49.9030489259, 49.7311328914, 49.5720305662, 49.4348004037, 49.3274012178, 49.2557369849, 49.2240119073, 49.2341730218, 49.2856634087, 49.3753903493, 49.4984793217, 49.6477395839, 49.8149788993, 49.9903231407, 50.1638047958, 50.3254436845, 50.4656417010, 50.5761703192, 50.6507464617, 50.6845702011, 50.6755781434, 50.6239365352, 50.5327506614, 50.4068739110, 50.2534807525, 50.0811244338, 49.8996654602, 49.7195882384, 49.5506916839, 49.4027121850, 49.2842800817, 49.2018092404, 49.1601074145, 49.1613683242, 49.2055441020, 49.2903971052, 49.4108077950, 49.5598535055, 49.7294382886, 49.9094142311, 50.0893278934, 50.2588773554, 50.4082336380, 50.5282822352, 50.6120316889, 50.6546682588, 50.6529888605, 50.6071124556, 50.5191789826, 50.3946215177, 50.2394290358, 50.0632446936, 49.8752755245, 49.6864538482, 49.5074758921, 49.3481861343, 49.2177724481, 49.1233337765, 49.0700915353, 49.0612416967, 49.0969008631, 49.1753493742, 49.2916209868, 49.4392803598, 49.6094963661, 49.7929558834, 49.9785769597, 50.1554320755, 50.3132290879, 50.4427423228, 50.5359987396, 50.5872868550, 50.5936948633, 50.5538058337, 50.4698576622, 50.3464962622, 50.1902063129, 50.0100587461, 49.8155164635, 49.6179343989, 49.4278477872, 49.2565223886, 49.1128660282, 49.0050664714, 48.9387619881, 48.9178427601, 48.9429343161, 49.0127711833, 49.1230007249, 49.2673333921, 49.4369207976, 49.6219949017, 49.8118842676, 49.9951413721, 50.1609693442, 50.2995251314, 50.4021146510, 50.4624810201, 50.4764806452, 50.4432456069, 50.3636294416, 50.2418939080, 50.0846288467]
# V1z = [0.0000501518, 0.0000054112, -0.0000674888, -0.0001183603, -0.0000363443, -0.0001683545, -0.0000644518, -0.0000783601, 0.0000058015, 0.0000240932, -0.0000762637, -0.0001518544, -0.0000439140, 0.0000067876, -0.0001173456, -0.0000514830, 0.0000028033, -0.0001068506, -0.0001578204, -0.0000456407, -0.0000426464, -0.0000516218, -0.0000843131, -0.0000873868, -0.0000531877, -0.0000644365, -0.0000162667, 0.0000074601, -0.0000301556, 0.0000362263, -0.0001420566, -0.0000382075, -0.0000030503, -0.0000548409, -0.0001608866, -0.0001112110, -0.0000748607, -0.0000291597, -0.0000789488, -0.0001506550, -0.0000126388, -0.0000624698, 0.0000748643, -0.0001391365, -0.0000306055, -0.0002133008, 0.0000205150, 0.0000461139, -0.0001245204, 0.0000688843, -0.0000691877, 0.0000781797, -0.0000890082, -0.0000856917, -0.0000831088, 0.0000326966, -0.0000199604, -0.0001104744, -0.0001710285, -0.0000716846, -0.0001135678, -0.0001006570, -0.0001409900, -0.0000029296, -0.0000444414, -0.0000226763, -0.0000950284, -0.0001677214, 0.0000575687, 0.0001381331, -0.0000445882, -0.0001098974, -0.0001468131, 0.0000058342, -0.0000606551, -0.0000576714, -0.0000233863, -0.0000750636, -0.0001336259, -0.0000137718, -0.0000986178, -0.0000681299, -0.0000808068, -0.0001263289, -0.0000243147, -0.0000793018, 0.0000272996, -0.0001167369, 0.0000435157, -0.0001835554, -0.0000756900, -0.0001219227, -0.0001802494, -0.0000497424, -0.0001619277, -0.0000671918, -0.0000593728, -0.0000966377, -0.0000403430, -0.0000865064, -0.0001108655, -0.0000956876, -0.0000484709, -0.0000622905, -0.0000253020, -0.0000611930, -0.0001264061, -0.0001478101, -0.0001217612, -0.0001051501, -0.0000941747, -0.0000050057, -0.0001329239, -0.0001638134, -0.0000800141, -0.0000866571, -0.0000737943, -0.0001078471, -0.0000484188, -0.0001224171, -0.0001675673, 0.0000054728, -0.0000482664, -0.0000717834, -0.0001226896, 0.0000856618, -0.0000119274, -0.0000489886, -0.0001283676, -0.0001167644, -0.0001322487, -0.0001660530, -0.0000291689, -0.0000397726, -0.0000750766, 0.0000478723, -0.0000149486, -0.0002086281, -0.0000738834, 0.0000720993, -0.0002219164, -0.0000181719, -0.0001599534, -0.0000480826, -0.0000463526, -0.0000345476, -0.0001497041, 0.0000036868, 0.0000317307, -0.0001445824, -0.0001881464, -0.0001853256, -0.0001611944, -0.0001977787, -0.0001098982, -0.0000882103, -0.0000718875, -0.0000697872, 0.0000173508, -0.0000785236, -0.0000137329, -0.0000354666, -0.0001013550, -0.0000579084, -0.0000847072, -0.0000984168, -0.0000506148, -0.0000262412, -0.0001227082, -0.0000690412, -0.0001183692, -0.0000784109, -0.0001287259, -0.0000957137, -0.0000852893, 0.0000055236, -0.0001031367, -0.0000504663, -0.0000785624, -0.0000631345, 0.0000023989, -0.0000936923, 0.0000280878, -0.0000259215, -0.0000105713, -0.0000568479, -0.0000038280, -0.0000177627, -0.0002222644, -0.0001298689, -0.0000543032, -0.0000723596, -0.0000538569, -0.0001353115, -0.0000869987, -0.0000789890, 0.0000503825, -0.0001403078, -0.0000373829, -0.0000027275, -0.0002116538, 0.0000147976, -0.0001051897, -0.0002093462, -0.0001709863, -0.0001801043, 0.0000065414, -0.0000462178, -0.0000944173, -0.0001374212, -0.0000255970, 0.0000197983, -0.0000904379, 0.0000121794, -0.0000075989, -0.0000958653, 0.0000320668, -0.0000883718, -0.0000559047, -0.0001520986, -0.0000809861, -0.0001441450, -0.0000531836, 0.0000157163, -0.0000594988, -0.0001060307, -0.0000558367, -0.0000625849, -0.0001284725, 0.0000737346, -0.0001611256, -0.0000613432, 0.0001451314, -0.0000391390, -0.0000864106, 0.0000136893, -0.0000829314, -0.0000837269, -0.0000819703, -0.0000633682, -0.0001071757, -0.0000247818, -0.0000085174, -0.0001040182, -0.0000865508, -0.0001330139, -0.0000780659, -0.0000814582, -0.0000489428, -0.0000163150, 0.0000019773, -0.0000270342, 0.0000290964, -0.0001383574, -0.0000383316, -0.0001495404, -0.0000364442, 0.0000020239, -0.0000629377, -0.0000728029, 0.0000030873, 0.0000338446, 0.0000553335, -0.0000368472, -0.0001022721, -0.0002233808, -0.0000190833, -0.0000831080, -0.0000832803, -0.0000938644, 0.0000178113, -0.0000990796, -0.0001266828, -0.0000578954, -0.0000176120, 0.0000276990, -0.0001353390, -0.0001086378, -0.0001050296, -0.0000096739, -0.0001737568, -0.0001728961, 0.0000044001, -0.0000516001, -0.0000736253, -0.0000653022, -0.0000901412, -0.0001794101, -0.0000642222, 0.0000013112, -0.0000487940, -0.0000940525, 0.0001053407, -0.0001394937, -0.0000367453, -0.0001302536, -0.0000205794, -0.0000259212, 0.0000077421, -0.0001148656, -0.0000604264, -0.0000293573, -0.0000406783, 0.0000465911, 0.0000537748, 0.0000512640, -0.0000352510, -0.0000133754, 0.0000835090, -0.0001042647, -0.0001007661, -0.0001335559, -0.0001903963, -0.0001211911, -0.0000001513, -0.0000049746, -0.0000615821, 0.0000148798, -0.0001085115, -0.0000265621, -0.0000863060, -0.0000978187, -0.0000268408, -0.0000627851, -0.0000574607, -0.0000971409, -0.0001680503, -0.0000427608, -0.0001189907, 0.0000513012, -0.0000525374, 0.0000915362, -0.0000605554, -0.0000488551, -0.0001339132, -0.0000616429, -0.0001081851, -0.0000837566, -0.0000816586, -0.0000644951, 0.0000595710, -0.0001319500, -0.0002305126, -0.0000791778, -0.0000464789, -0.0001145428, -0.0000122241, -0.0000783101, -0.0000086775, 0.0000808231, -0.0000236813, -0.0000165992, -0.0000881681, -0.0000371508, -0.0000323275, -0.0001408626, -0.0000792960, -0.0002091055, -0.0001464440, -0.0000632905, 0.0000509678, -0.0000149614, -0.0001083439, -0.0001556283, 0.0000267333, -0.0001397224, -0.0000827058, -0.0001342235, -0.0001043679, -0.0001307354, 0.0000131373, -0.0000259270, 0.0001145636, -0.0000581818, -0.0001048450, -0.0001513473, -0.0000334324, -0.0000193515, -0.0000583536, -0.0000853573, -0.0001402845, -0.0000289310, -0.0001216607, 0.0000354416, 0.0000454875, -0.0000798553, -0.0000803860, -0.0001583103, 0.0000138489, -0.0000412256, -0.0001398112, -0.0001075873, -0.0000555020, 0.0000051076, -0.0000958028, -0.0001032330, -0.0000635758, 0.0000014000, -0.0001742051, -0.0001713596, -0.0000736397, -0.0000578310, -0.0001382322, -0.0000164535, 0.0000132036, 0.0000514910, 0.0000113418, -0.0000371048, -0.0001811284, 0.0000630829, -0.0000495667, 0.0000074565, 0.0000006012, -0.0000227851, -0.0000327322, -0.0000418451, -0.0001083225, -0.0001498515, -0.0000682041, -0.0001255440, -0.0001335645, -0.0001341655, 0.0000315573, -0.0000269365, -0.0001326317, -0.0001191303, -0.0001318565, -0.0000692346, -0.0000501210, 0.0000177122, 0.0000174036, -0.0001480929, -0.0001825354, -0.0000645650, -0.0000711464, -0.0001110295, -0.0000711139, 0.0000060671, -0.0000698408, -0.0000874498, -0.0000195258, -0.0000311545, -0.0000938094, -0.0001584335, 0.0000027103, -0.0001223044, -0.0000115732, 0.0000360417, 0.0000285025, -0.0001737280, -0.0000611977, -0.0000517272, -0.0000074815, -0.0000433692, -0.0001415051, -0.0001685066, -0.0001642514, -0.0002071748, -0.0002806963, -0.0000153372, -0.0001513475, -0.0000878950, 0.0000154171, 0.0000107744, -0.0000754305, -0.0000303738, -0.0000478445, -0.0000241737, -0.0000237216, -0.0001145550, -0.0000452322, -0.0001173089, -0.0001378878, -0.0000361916, -0.0000134562, -0.0000880990, -0.0000875988, -0.0002389569, -0.0000310449, -0.0000207070, -0.0000537353, -0.0000223377, -0.0001196704, -0.0001393563, -0.0000147177, 0.0000711436, -0.0001645506, -0.0001103931, -0.0000905359, -0.0001758935, -0.0000948605, -0.0001119793, -0.0001122130, -0.0001225373, -0.0001077138, -0.0000919206, 0.0000411679, 0.0000446773, 0.0000382028, -0.0000761796, -0.0000714331, -0.0000237871, -0.0001487988, -0.0001175940, -0.0002547712, -0.0000283948, -0.0000703904, -0.0001075451, -0.0001430570, -0.0000056752, -0.0001033679, -0.0001914005, -0.0000377826, -0.0000918940, -0.0001288042, -0.0000659300, -0.0001330711, 0.0000400896, -0.0000927430, -0.0001458164, -0.0000225456, -0.0001313638, -0.0001579534, -0.0002222178, -0.0000183170, -0.0000690750, -0.0001395717, -0.0001825132, -0.0001497508, -0.0001153608, -0.0001892758, -0.0000042538, -0.0001363445, -0.0001081246, -0.0002452405, -0.0001243469, 0.0000531683, -0.0000732411, -0.0003013667, -0.0001179878, -0.0000357014, -0.0000288589, -0.0000359981, -0.0001215961, -0.0001324152, -0.0000685502, -0.0000317746, -0.0000573053, -0.0000944800, -0.0000942643, -0.0002024353, -0.0001482414, -0.0001090009, 0.0000059799, 0.0000275382, -0.0002604458, -0.0002152756, 0.0000057113, -0.0001295558, -0.0000961428, -0.0001682873, -0.0001246381, -0.0002023190, -0.0001195338, -0.0000615683, -0.0000461189, 0.0000039352, -0.0000858411, -0.0000171324, -0.0001144765, -0.0000785377, -0.0000182415, -0.0000551933, -0.0001096112, -0.0001100980, 0.0000735254, -0.0000879763, -0.0000600991, -0.0000498088, -0.0001097006, -0.0001166370, -0.0000527758, -0.0001023212, -0.0000222253, -0.0000940947, 0.0000765293, -0.0001592018, -0.0000053413, -0.0000886607, -0.0001474906, -0.0000406233, -0.0000792488, -0.0000586114, 0.0000129297, -0.0000960049, 0.0000784756, -0.0001278572, -0.0000614397, -0.0000935924, -0.0000884726, -0.0000916828, -0.0002077347, -0.0000305506, -0.0000369926, -0.0000147155, -0.0000721379, -0.0000692529, -0.0000772715, -0.0000638545, -0.0000019895, -0.0000668776, 0.0000210365, 0.0000030603, -0.0003268305, -0.0001942241, -0.0000328662, -0.0001491712, -0.0001226438, -0.0000782557, -0.0001008796, -0.0000458036, 0.0001620942, -0.0001689360, -0.0001231021, -0.0000714068, -0.0001472875, -0.0000429679, -0.0000550140, -0.0000932074, -0.0001612277, 0.0000300587, -0.0001111600, -0.0001391537, -0.0000690889, -0.0001390243, -0.0000850395, -0.0000224151, 0.0000043565, 0.0000283976, -0.0000433970, -0.0001170664, -0.0000664760, -0.0000183800, -0.0001511788, -0.0001462894, -0.0001008521, -0.0000874306, -0.0002314191, 0.0000094086, -0.0000424855, -0.0001349459, -0.0000507125, -0.0001612702, -0.0001962155, -0.0001568570, 0.0000105166, 0.0000187758, 0.0000016806, -0.0000624306, -0.0000492348, -0.0000342398, -0.0001257380, -0.0000410677, -0.0000395101, -0.0000251385, -0.0000616773, -0.0001488279, -0.0000659217, -0.0000855177, 0.0000026190, -0.0000598294, -0.0001266071, -0.0000127240, -0.0000822997, -0.0000252448, -0.0000782369, 0.0000162298, -0.0001673378, -0.0000917794, 0.0000395231, 0.0000125905, 0.0000081890, -0.0000789077, -0.0001125536, -0.0000791728, -0.0000797998, -0.0001413966, -0.0001574127, -0.0000733535, -0.0000973713, -0.0000700712, -0.0000350767, -0.0000497480, -0.0001709505, -0.0000459963, -0.0001758338, 0.0000021405, -0.0001993761, -0.0001009545, -0.0001954589, -0.0000785427, -0.0000639958, -0.0000356146, 0.0000621417, 0.0000519665, -0.0000368858, -0.0000259387, -0.0000694048, 0.0000188580, -0.0000541364, -0.0001004682, 0.0000563422, -0.0000795836, -0.0001228087, 0.0000307129, -0.0000481137, -0.0001607922, 0.0000020095, -0.0000348943, -0.0001888187, 0.0000328668, -0.0000237022, -0.0000913908, -0.0001034042, 0.0000058961, 0.0000223987, -0.0001178838, -0.0000067646, -0.0001090281, 0.0000732972, -0.0000837147, -0.0001529268, 0.0000002590, -0.0000960187, -0.0001149378, -0.0000623449, -0.0000956957, -0.0000686182, -0.0000669789, -0.0000359587, 0.0000159328, -0.0001083977, -0.0001432179, -0.0001521877, -0.0000899686, -0.0001271266, -0.0001435985, -0.0001071915, 0.0000294975, -0.0000889283, -0.0000050407, 0.0000035738, -0.0001160591, -0.0001686679, -0.0001681824, -0.0000271771, -0.0001584996, -0.0001077793, -0.0000518033, -0.0002053449, -0.0000475317, -0.0000561742, -0.0000544217, -0.0000508829, -0.0002001391, -0.0000639619, 0.0000016950, -0.0000953643, -0.0000507241, -0.0001645264, -0.0001043696, -0.0001197401, -0.0001671085, -0.0000402817, -0.0001554781, -0.0000489625, -0.0001095479, -0.0000404676, -0.0000291339, -0.0000393841, -0.0000553974, 0.0000012085, -0.0000929637, -0.0000816430, -0.0000752332, -0.0000719233, -0.0002452763, 0.0000038947, -0.0001180150, 0.0000773193, -0.0000335212, -0.0000583594, -0.0000910585, -0.0001642683, -0.0000526468, -0.0000203418, -0.0001084685, -0.0000734926, 0.0000936368, 0.0000294609, -0.0000134223, -0.0000727299, -0.0000349934, 0.0000318698, -0.0000969436, -0.0000906505, -0.0001220346, -0.0001039882, -0.0000510433, -0.0000465594, -0.0001269598, -0.0001111913, -0.0000123335, -0.0001331385, -0.0000997122, -0.0000672129, -0.0000119418, -0.0000713607, -0.0000681127, -0.0000276865, -0.0001036909, 0.0000004924, -0.0000951786, 0.0000132611, -0.0001581990, 0.0000751110, -0.0000811815, -0.0000442948, -0.0000157480, -0.0001201453, -0.0002453088, -0.0001439548, 0.0000538666, -0.0001943451, 0.0000276684, -0.0000761077, -0.0001621653, 0.0000227837, 0.0000049383, -0.0000630862, 0.0000362030, -0.0001202627, 0.0000175596, -0.0000899511, -0.0001678249, -0.0000788977, -0.0001070721, 0.0000513997, 0.0000475325, -0.0001613162, -0.0001462999, 0.0000446018, -0.0001463782, -0.0001598716, -0.0000659123, -0.0001427808, -0.0001384121, -0.0001429828, -0.0000183500, 0.0000158654, -0.0001458769, -0.0001121006, -0.0000241154, -0.0000123127, -0.0000751371, 0.0000110836, -0.0000914147, -0.0000569944, -0.0001129191, -0.0000338105, -0.0001415329, -0.0001945832, -0.0000656930, -0.0001833650, -0.0000143105, -0.0000623204, -0.0000537712, -0.0001076828, -0.0000928893, -0.0001060749, -0.0000119110, 0.0000250702, -0.0001034509, -0.0001522313, -0.0001013545, -0.0000225936, -0.0000787834, -0.0001606118, -0.0000793637, -0.0000718278, -0.0001171011, -0.0001007097, -0.0000678399, -0.0000528233, -0.0001801317, -0.0000347988, -0.0000732867, 0.0000088815, -0.0001327255, -0.0000855380, 0.0000095901, -0.0000622591, -0.0000201425, -0.0000522576, -0.0002049677, -0.0001157952, -0.0000129333, -0.0001768191, -0.0000131262, 0.0000205383, -0.0000588277, -0.0001697104, 0.0000118106, -0.0000041303, -0.0000787109, -0.0001337325, -0.0001600439, -0.0000926864, -0.0000044501, -0.0002378531, 0.0000039899, -0.0001171868, -0.0000922445, -0.0001491965, -0.0000930142, -0.0000477746, -0.0000372583, -0.0000967776, -0.0000292863, -0.0000559289, 0.0000249833, -0.0001390091, -0.0001848532, -0.0000173150, -0.0001729406, -0.0001351375, -0.0000852463, 0.0000551428, -0.0002129916, -0.0000555245, -0.0000009787, -0.0001131714, -0.0000906814, 0.0000618411, -0.0002245559, -0.0000891853, -0.0000596098, -0.0001182199, -0.0000825509, -0.0000180037, -0.0002060438, -0.0000923789, -0.0000709960, -0.0000452932, -0.0000185824, -0.0001658393, -0.0001107103, -0.0000924549, -0.0002211020, 0.0000356914, -0.0001308700, 0.0000219411, 0.0000521597, -0.0000395328, 0.0000200492, 0.0000051134, -0.0001964837, 0.0000038100, -0.0001998968, -0.0000219277, -0.0002010309, -0.0000476746, -0.0001789602, -0.0002035919, -0.0000776318, -0.0000662076, 0.0000563241, -0.0000320649, -0.0001128697, -0.0000926853, 0.0000011360, -0.0000246589, 0.0000021794, -0.0000443484, -0.0001231473, -0.0000687800, -0.0000914208, 0.0000263267, -0.0000516176, -0.0000212391, -0.0000862860, -0.0001151909, -0.0001344053, -0.0001798761, 0.0000300793, -0.0000045358, -0.0000996675, -0.0000974390, -0.0000649933, -0.0000679742]

# ─────────────────────────────────────
# 4. GRAPE optimization (finite-diff grad)
# ─────────────────────────────────────
history=[]
for ep in range(n_epochs+1):
    # learning-rate schedule
    lr = 5e+1 if ep<=10000000 else 1e-6

    if ep>0:
        f0_base,_ = f0_and_trace(V1x,V1y,V1z,
                                 theta_rot,tau,N_slices,
                                 B0,omega0,h_FD,J,projs)
        grads=[]
        eps=1e-5
        for arr in (V1x,V1y,V1z):
            g=np.zeros_like(arr)
            for i in range(N_slices):
                arr[i]+=eps
                f_pert,_=f0_and_trace(V1x,V1y,V1z,
                                      theta_rot,tau,N_slices,
                                      B0,omega0,h_FD,J,projs)
                g[i]=(f_pert-f0_base)/eps
                arr[i]-=eps
            grads.append(g)
        V1x+=lr*grads[0]; V1y+=lr*grads[1]; V1z+=lr*grads[2]

    f0_val,tr_inv=f0_and_trace(V1x,V1y,V1z,
                               theta_rot,tau,N_slices,
                               B0,omega0,h_FD,J,projs)
    history.append(tr_inv)
    if ep%10==0:
        print(f"Ep {ep:04d}: f0={f0_val:.6e}  Tr[Fcl^-1]={tr_inv:.10e}")

# ─────────────────────────────────────
# 5. Results
# ─────────────────────────────────────
print("\n# Optimised control fields:")
print("V1x = " + "[" + ", ".join(f"{v:.10f}" for v in V1x) + "]")
print("V1y = " + "[" + ", ".join(f"{v:.10f}" for v in V1y) + "]")
print("V1z = " + "[" + ", ".join(f"{v:.10f}" for v in V1z) + "]")

plt.plot(history); plt.xlabel("epoch"); plt.ylabel("Tr[F_cl^-1]")
plt.title("Training progress (B, ω)"); plt.grid(); plt.show()